# Run before

In [27]:
library(tidyverse)
library(repr)
library(tidymodels)

# Download files

In [56]:
sat_scores = read_csv("https://raw.githubusercontent.com/emi31770/proposal/main/datesets/sat_scores.csv")
zip_income = read_csv("https://raw.githubusercontent.com/emi31770/proposal/main/datesets/zipcodes_income.csv")
names(sat_scores) = make.names(names(sat_scores))
names(zip_income) = make.names(names(zip_income))
write_csv(sat_scores, "sat.csv")
write_csv(zip_income, "zip.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  `Zip Code` = col_double(),
  Latitude = col_double(),
  Longitude = col_double(),
  `Start Time` = col_time(format = ""),
  `End Time` = col_time(format = ""),
  `Student Enrollment` = col_double(),
  `Average Score (SAT Math)` = col_double(),
  `Average Score (SAT Reading)` = col_double(),
  `Average Score (SAT Writing)` = col_double()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  state = col_character(),
  zipcode = col_double(),
  total_pop = col_double(),
  total_income = col_double(),
  country = col_character(),
  avg_income = col_double()
)



In [81]:
zip_income = read_csv("zip.csv")
sat_scores = read_csv("sat.csv")

zip_income = zip_income %>% filter(state=="NY", country=="USA") %>% select(zipcode, total_pop, total_income, avg_income)

sat_scores = sat_scores %>% mutate(Averages.Sum = Average.Score..SAT.Math. + Average.Score..SAT.Reading. + Average.Score..SAT.Writing.) %>%
                            select(Zip.Code, Student.Enrollment, Averages.Sum, Average.Score..SAT.Math., Average.Score..SAT.Reading., Average.Score..SAT.Writing.) %>%
                            na.omit() %>%
                            group_by(Zip.Code) %>%
                            summarize_all("mean")

head(sat_scores, 5)
head(zip_income, 5)

Parsed with column specification:
cols(
  state = col_character(),
  zipcode = col_double(),
  total_pop = col_double(),
  total_income = col_double(),
  country = col_character(),
  avg_income = col_double()
)

Parsed with column specification:
cols(
  .default = col_character(),
  Zip.Code = col_double(),
  Latitude = col_double(),
  Longitude = col_double(),
  Start.Time = col_time(format = ""),
  End.Time = col_time(format = ""),
  Student.Enrollment = col_double(),
  Average.Score..SAT.Math. = col_double(),
  Average.Score..SAT.Reading. = col_double(),
  Average.Score..SAT.Writing. = col_double()
)

See spec(...) for full column specifications.



Zip.Code,Student.Enrollment,Averages.Sum,Average.Score..SAT.Math.,Average.Score..SAT.Reading.,Average.Score..SAT.Writing.
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10002,504.7273,1358.909,472.9091,448.5455,437.4545
10003,693.2000,1323.600,449.6000,443.4000,430.6000
10004,519.2500,1380.250,463.0000,462.0000,455.2500
10006,747.5000,1268.000,429.5000,419.0000,419.5000
10009,666.0000,1327.000,454.0000,428.0000,445.0000


zipcode,total_pop,total_income,avg_income
<dbl>,<dbl>,<dbl>,<dbl>
0,9632130,887473952,92136.83
10001,16150,2906435,179965.02
10002,42400,2718913,64125.31
10003,29270,8191737,279868.02
10004,2470,944925,382560.73
